In [1]:
WORKER_ID = 1

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(WORKER_ID)

In [3]:
# whether to log each feature and sequence status
verbose = True

In [4]:
import gc
import os
import pandas as pd
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [5]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [6]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [7]:
from deepvideoclassification.architectures import Architecture

Using TensorFlow backend.


# Run experiments

In [8]:
experiment_batch_name = 'experiment_batch_1'

In [9]:
# load list of experiments
experiments = pd.read_csv(pwd + "experiments/" + experiment_batch_name + '.csv')

In [10]:
print(experiments.shape)

(5376, 12)


In [ ]:
###################
### Run experiments
###################

for row in experiments.values:
    
    # get experiment params from dataframe row
    experiment = dict(zip(experiments.columns, row))
    
    # only run experiment if not already run
    if not os.path.exists(pwd + 'models/' + str(experiment["model_id"]) + '/results.json'):

        # only run experiment if matches this worker id
        if experiment['WORKER'] == WORKER_ID:
            
            print(str(experiment["model_id"]) + "   " + "X"*60)
            logging.info("Begin experiment for model_id={} on GPU:{} ".format(experiment['model_id'], os.environ["CUDA_VISIBLE_DEVICES"]))
            print(experiment)

            architecture = Architecture(model_id = experiment['model_id'], 
                                        architecture = experiment['architecture'], 
                                        sequence_length = experiment['sequence_length'], 
                                        pretrained_model_name = experiment['pretrained_model_name'],
                                        pooling = experiment['pooling'],
                                        sequence_model = experiment['sequence_model'],
                                        sequence_model_layers = experiment['sequence_model_layers'],
                                        layer_1_size = experiment['layer_1_size'],
                                        layer_2_size = experiment['layer_2_size'],
                                        layer_3_size = experiment['layer_3_size'],
                                        dropout = experiment['dropout'],
                                        verbose=True)

            architecture.train_model()
            
            gc.collect()

2019-01-20 13:23:17,765 [MainThread  ] [INFO ]  Begin experiment for model_id=81 on GPU:1 
2019-01-20 13:23:17,766 [MainThread  ] [INFO ]  Model folder exists but no results found - potential error in previous model training
2019-01-20 13:23:17,768 [MainThread  ] [INFO ]  Loading data


81   XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
{'layer_2_size': 512, 'WORKER': 1, 'layer_3_size': 256, 'layer_1_size': 256, 'model_id': 81, 'architecture': 'image_MLP_frozen', 'sequence_model': nan, 'sequence_model_layers': nan, 'sequence_length': 1.0, 'pretrained_model_name': 'vgg16', 'pooling': 'max', 'dropout': 0.2}


2019-01-20 13:23:18,703 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/vgg16/max/
2019-01-20 13:23:18,704 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]


Done initializing data with #samples: train=60597, valid=6416, test=3139
Train on 60597 samples, validate on 6416 samples
Epoch 1/20
60597/60597 [==============================] - 20s 326us/step - loss: 0.1823 - acc: 0.9354 - val_loss: 0.1916 - val_acc: 0.9263

Epoch 00001: val_acc improved from -inf to 0.92628, saving model to /mnt/seals/models/81/model_round_1.h5
Epoch 2/20
60597/60597 [==============================] - 14s 223us/step - loss: 0.1267 - acc: 0.9502 - val_loss: 0.1798 - val_acc: 0.9257

Epoch 00002: val_acc did not improve from 0.92628
Epoch 3/20
60597/60597 [==============================] - 13s 222us/step - loss: 0.1178 - acc: 0.9533 - val_loss: 0.2360 - val_acc: 0.9110

Epoch 00003: val_acc did not improve from 0.92628
Epoch 4/20
60597/60597 [==============================] - 13s 222us/step - loss: 0.1115 - acc: 0.9561 - val_loss: 0.1939 - val_acc: 0.9271

Epoch 00004: val_acc improved from 0.92628 to 0.92706, saving model to /mnt/seals/models/81/model_round_1.h5
Epo

2019-01-20 13:28:18,776 [MainThread  ] [INFO ]  {'layer_2_size': 512, 'fit_dt_train_end': '2019-01-20 13:28:16', 'fit_val_acc': 0.9280593353613951, 'fit_val_loss': 0.22092746094129329, 'fit_best_round': 3, 'fit_dt_train_start': '2019-01-20 13:23:19', 'fit_train_acc': 0.972763867615839, 'data_total_rows_valid': 6416, 'batch_size': 32, 'data_total_rows_test': 3139, 'frame_size': (224, 224), 'sequence_model_layers': nan, 'fit_dt_test_start': '2019-01-20 13:28:16', 'fit_stopped_epoch1': 8, 'num_features': 512, 'fit_test_acc': 0.6578528193692259, 'model_weights_path': None, 'dropout': 0.2, 'convolution_kernel_size': 3, 'path_model': '/mnt/seals/models/81/', 'layer_3_size': 256, 'layer_1_size': 256, 'data_total_rows_train': 60597, 'model_id': 81, 'fit_train_loss': 0.06430916750841183, 'architecture': 'image_mlp_frozen', 'sequence_model': nan, 'fit_dt_test_end': '2019-01-20 13:28:17', 'sequence_length': 1.0, 'fit_stopped_epoch2': 3, 'pretrained_model_name': 'vgg16', 'pooling': 'max', 'fit_num

...

Done initializing data with #samples: train=60597, valid=6416, test=3139
Train on 60597 samples, validate on 6416 samples
Epoch 1/20
60597/60597 [==============================] - 20s 331us/step - loss: 0.1742 - acc: 0.9349 - val_loss: 0.1601 - val_acc: 0.9324

Epoch 00001: val_acc improved from -inf to 0.93240, saving model to /mnt/seals/models/137/model_round_1.h5
Epoch 2/20
60597/60597 [==============================] - 17s 277us/step - loss: 0.1208 - acc: 0.9521 - val_loss: 0.1581 - val_acc: 0.9357

Epoch 00002: val_acc improved from 0.93240 to 0.93574, saving model to /mnt/seals/models/137/model_round_1.h5
Epoch 3/20
60597/60597 [==============================] - 17s 281us/step - loss: 0.1111 - acc: 0.9555 - val_loss: 0.1617 - val_acc: 0.9315

Epoch 00003: val_acc did not improve from 0.93574
Epoch 4/20
60597/60597 [==============================] - 17s 273us/step - loss: 0.1053 - acc: 0.9583 - val_loss: 0.1627 - val_acc: 0.9294

Epoch 00004: val_acc did not improve from 0.93574
E

2019-01-20 13:59:18,594 [MainThread  ] [INFO ]  {'layer_2_size': 128, 'fit_dt_train_end': '2019-01-20 13:59:14', 'fit_val_acc': 0.935295707864357, 'fit_val_loss': 0.16730914196171368, 'fit_dt_train_start': '2019-01-20 13:54:50', 'data_total_rows_valid': 6416, 'batch_size': 32, 'data_total_rows_test': 3139, 'frame_size': (224, 224), 'sequence_model_layers': nan, 'fit_dt_test_start': '2019-01-20 13:59:16', 'fit_stopped_epoch1': 2, 'num_features': 2048, 'fit_test_acc': 0.6683657215673782, 'model_weights_path': None, 'dropout': 0.2, 'convolution_kernel_size': 3, 'path_model': '/mnt/seals/models/137/', 'layer_3_size': 256, 'layer_1_size': 512, 'data_total_rows_train': 60597, 'fit_best_round': 3, 'model_id': 137, 'fit_train_loss': 0.0705202682573939, 'fit_train_acc': 0.9712032058706126, 'architecture': 'image_mlp_frozen', 'sequence_model': nan, 'fit_dt_test_end': '2019-01-20 13:59:17', 'sequence_length': 1.0, 'fit_stopped_epoch2': 1, 'pretrained_model_name': 'resnet50', 'pooling': 'max', 'fi

145   XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
{'layer_2_size': 512, 'WORKER': 1, 'layer_3_size': 256, 'layer_1_size': 256, 'model_id': 145, 'architecture': 'image_MLP_frozen', 'sequence_model': nan, 'sequence_model_layers': nan, 'sequence_length': 1.0, 'pretrained_model_name': 'resnet50', 'pooling': 'max', 'dropout': 0.2}


2019-01-20 13:59:20,326 [MainThread  ] [INFO ]  Features already cached: /mnt/seals/cache/features/resnet50/max/
2019-01-20 13:59:20,327 [MainThread  ] [INFO ]  Loading features data into memory [may take a few minutes]


Done initializing data with #samples: train=60597, valid=6416, test=3139
Train on 60597 samples, validate on 6416 samples
Epoch 1/20
60597/60597 [==============================] - 19s 317us/step - loss: 0.1589 - acc: 0.9391 - val_loss: 0.2001 - val_acc: 0.9177

Epoch 00001: val_acc improved from -inf to 0.91766, saving model to /mnt/seals/models/145/model_round_1.h5
Epoch 2/20
60597/60597 [==============================] - 15s 252us/step - loss: 0.1213 - acc: 0.9517 - val_loss: 0.1509 - val_acc: 0.9419

Epoch 00002: val_acc improved from 0.91766 to 0.94193, saving model to /mnt/seals/models/145/model_round_1.h5
Epoch 3/20
60597/60597 [==============================] - 16s 257us/step - loss: 0.1129 - acc: 0.9554 - val_loss: 0.1635 - val_acc: 0.9356

Epoch 00003: val_acc did not improve from 0.94193
Epoch 4/20
60597/60597 [==============================] - 16s 261us/step - loss: 0.1088 - acc: 0.9572 - val_loss: 0.1453 - val_acc: 0.9448

Epoch 00004: val_acc improved from 0.94193 to 0.944